## ReMECS -- Real-time Multimodal Emotion Classification Syatem

## Using multimodal data stream [EEG+EDA+RESP_BELT] 

## Using RNN + SGD + One pass 

## Modifications:
- Reward Penalty based Weighted Ensemble [RPWE]


## Date: 23 Sept 2021 at 10:10 AM

In [8]:
#============================
# Import important libraries
#============================
import pandas as pd 
import numpy as np
import tensorflow as tf
import math
import scipy
import pywt
from creme import metrics
import time
import datetime
from statistics import mode
from scipy import stats
from sklearn import preprocessing
from collections import defaultdict, Counter
from scipy.special import expit
from mlxtend.plotting import plot_confusion_matrix
import matplotlib.pyplot as plt

from window_slider import Slider
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import SimpleRNN
from keras.layers import Dropout

%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

# from IPython.core.display import display, HTML
# display(HTML("<style>.container { width:100% !important; }</style>"))

In [9]:
def calculate_entropy(list_values):
    counter_values = Counter(list_values).most_common()
    probabilities = [elem[1]/len(list_values) for elem in counter_values]
    entropy=scipy.stats.entropy(probabilities)
    return entropy

def calculate_statistics(list_values):
    n5 = np.nanpercentile(list_values, 5)
    n25 = np.nanpercentile(list_values, 25)
    n75 = np.nanpercentile(list_values, 75)
    n95 = np.nanpercentile(list_values, 95)
    median = np.nanpercentile(list_values, 50)
    mean = np.nanmean(list_values)
    std = np.nanstd(list_values)
    var = np.nanvar(list_values)
    rms = np.nanmean(np.sqrt(list_values**2))
    return [n5, n25, n75, n95, median, mean, std, var, rms]

def calculate_crossings(list_values):
    zero_crossing_indices = np.nonzero(np.diff(np.array(list_values)> 0))[0]
    no_zero_crossings = len(zero_crossing_indices)
    mean_crossing_indices = np.nonzero(np.diff(np.array(list_values) > np.nanmean(list_values)))[0]
    no_mean_crossings = len(mean_crossing_indices)
    return [no_zero_crossings, no_mean_crossings]

def get_features(list_values):    
    list_values = list_values[0,:]
    entropy = calculate_entropy(list_values)
    crossings = calculate_crossings(list_values)
    statistics = calculate_statistics(list_values)
    return [entropy] + crossings + statistics

In [10]:
#======================================================
# EDA Feature Extraction (Wavelet Features)
#======================================================
def extract_eda_features(raw_eda):
    features =[]
    EDA = raw_eda
    list_coeff = pywt.wavedec(EDA, 'db4', level=3)
    
#     print(list_coeff)
    for coeff in list_coeff:
        features += get_features(coeff)
    return features


In [11]:
#======================================================
# RESP BELT Feature Extraction (Wavelet Features)
#======================================================

def extract_resp_belt_features(raw_data):
    features =[]
    resp_belt = raw_data
    list_coeff = pywt.wavedec(resp_belt, 'db4', level=3)
    
#     print(list_coeff)
    for coeff in list_coeff:
        features += get_features(coeff)
    return features

In [12]:
def eeg_features(raw_data):
    ch = 0
    features= []
    def calculate_entropy(list_values):
        counter_values = Counter(list_values).most_common()
        probabilities = [elem[1]/len(list_values) for elem in counter_values]
        entropy=scipy.stats.entropy(probabilities)
        return entropy

    def calculate_statistics(list_values):
        median = np.nanpercentile(list_values, 50)
        mean = np.nanmean(list_values)
        std = np.nanstd(list_values)
        var = np.nanvar(list_values)
        rms = np.nanmean(np.sqrt(list_values**2))
        return [median, mean, std, var, rms]

    def get_features(list_values):    
    #     list_values = list_values[0,:]
        entropy = calculate_entropy(list_values)
        statistics = calculate_statistics(list_values)
        return [entropy] + statistics
    
    for i in range(raw_data.shape[0]):
        ch_data = raw_data[i]
        list_coeff = pywt.wavedec(ch_data, 'db4', level=5)
        for coeff in list_coeff:
            features += get_features(coeff)
            
        ch = ch+1
    return features

In [13]:
# ##===================================================
# # EEG data read from files
# ##===================================================
# def eeg_data(p,v):
#     file_eeg = '/home/gp/Desktop/MER_arin/data/eeg_data/'+str(p)+'_data_DEAP'+'.csv'
#     print(file_eeg)
#     df = pd.read_csv(file_eeg,sep=',', header = None, engine='python')
#     eeg_sig = df.loc[df.iloc[:,1] == v]
#     return eeg_sig

# ##===================================================
# # EDA data read from files
# ##===================================================
# def eda_data(p,v):
#     file_eda = '/home/gp/Desktop/MER_arin/data/eda_data/'+str(p)+'_GSR_data_from_DEAP.csv'
#     print(file_eda)
#     df = pd.read_csv(file_eda,sep=',', header = None, engine='python')
#     eda_sig = df.loc[df.iloc[:,1] == v]
#     return eda_sig

# ##===================================================
# # Resp data read from files
# ##===================================================
# def resp_data(p,v):
#     file_resp = '/home/gp/Desktop/MER_arin/data/resp_data/'+str(p)+'_Respiration_data_from_DEAP.csv'
#     print(file_resp)
#     df = pd.read_csv(file_resp,sep=',', header = None, engine='python')
#     resp_sig = df.loc[df.iloc[:,1] == v]
#     return resp_sig

In [14]:
#==================================
# RNN model
#==================================
def create_model(x):
    model = Sequential() 
    
    model.add(SimpleRNN(units = 3, activation='sigmoid',return_sequences = True, input_shape = (1,x.shape[1])))

    model.add(SimpleRNN(units = 3, activation='sigmoid',return_sequences = True))
    
    model.add(Dense(1, activation='sigmoid'))
    
    opt = tf.keras.optimizers.SGD(learning_rate=0.05)
    
    model.compile(loss='binary_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    
    
    print(model.summary())
    
    return model

In [16]:
#================================================================
# Initialization of different parameters and performance metrics
#================================================================
segment_in_sec = 10 #in sec
bucket_size = int((8064/60)*segment_in_sec)  #8064 is for 60 sec record
overlap_count = 0

num_classifiers = 3 #Total number of classifiers
w_val =np.ones(num_classifiers)/num_classifiers #Weights for valence classifiers
w_aro =np.ones(num_classifiers)/num_classifiers #Weights for valence classifiers

cBbest_val = [] #Classifiers least error for valence classsification
cBest_aro = []  #Classifiers least error for arousal classsification
cWorst_val = [] #Classifiers heighest error for valence classsification
cWorst_aro = []  #Classifiers heighest error for arousal classsification

beta_val = np.ones(num_classifiers)/num_classifiers
beta_aro = np.ones(num_classifiers)/num_classifiers


optimizer= 'SGD' #optimizer
classifier = 'RNN_RPWE'+str(optimizer)
indx = 0
c=0


participant = 32
videos = 40

global eeg_emotion, eda_emotion, resp_emotion, mer_emotion, all_eta
eeg_emotion = []
eda_emotion = []
resp_emotion = []
mer_emotion = []

all_eta =[]
init_m = 0

#================================================
# Performance matric declaration here
#================================================
mer_acc_val = metrics.Accuracy() #Accuracy
mer_f1m_val = metrics.F1() #F1 measure  
mer_acc_aro = metrics.Accuracy() #Accuracy
mer_f1m_aro = metrics.F1() #F1 measure
mer_roc_val = metrics.ROCAUC()
mer_roc_aro = metrics.ROCAUC() 
mer_mcc_val = metrics.MCC()
mer_mcc_aro = metrics.MCC()


eeg_acc_val = metrics.Accuracy() #Accuracy
eeg_f1m_val = metrics.F1() #F1 measure  
eeg_acc_aro = metrics.Accuracy() #Accuracy
eeg_f1m_aro = metrics.F1() #F1 measure
eeg_roc_val = metrics.ROCAUC()
eeg_roc_aro = metrics.ROCAUC()
eeg_mcc_val = metrics.MCC()
eeg_mcc_aro = metrics.MCC()


eda_acc_val = metrics.Accuracy() #Accuracy
eda_f1m_val = metrics.F1() #F1 measure  
eda_acc_aro = metrics.Accuracy() #Accuracy
eda_f1m_aro = metrics.F1() #F1 measure
eda_roc_val = metrics.ROCAUC()
eda_roc_aro = metrics.ROCAUC()
eda_mcc_val = metrics.MCC()
eda_mcc_aro = metrics.MCC()


resp_acc_val = metrics.Accuracy() #Accuracy
resp_f1m_val = metrics.F1() #F1 measure  
resp_acc_aro = metrics.Accuracy() #Accuracy
resp_f1m_aro = metrics.F1() #F1 measure
resp_roc_val = metrics.ROCAUC()
resp_roc_aro = metrics.ROCAUC()
resp_mcc_val = metrics.MCC()
resp_mcc_aro = metrics.MCC()

val_MSE = []
aro_MSE = []

haha_val = []
haha_aro = []


 
mer_cm_val  = metrics.ConfusionMatrix()
mer_cm_aro  = metrics.ConfusionMatrix()

eeg_cm_val  = metrics.ConfusionMatrix()
eeg_cm_aro  = metrics.ConfusionMatrix()

eda_cm_val  = metrics.ConfusionMatrix()
eda_cm_aro  = metrics.ConfusionMatrix()
  
resp_cm_val  = metrics.ConfusionMatrix()
resp_cm_aro  = metrics.ConfusionMatrix()


    
itr = 0 #controls the learning rate


#=======================================
# MAIN Loop STARTS HERE
#=======================================
for ii in range(0,participant):

        p =ii+1 #participant number
        
        print('-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-')
        ##===================================================
        # EEG data read from files
        ##===================================================
        def eeg_data(p):
            file_eeg = '/home/gp/Desktop/MER_arin/DEAP_data/eeg_data/'+str(p)+'_data_DEAP'+'.csv'
            print(file_eeg)
            eeg_sig = pd.read_csv(file_eeg,sep=',', header = None, engine='python')
            return eeg_sig

        ##===================================================
        # EDA data read from files
        ##===================================================
        def eda_data(p):
            file_eda = '/home/gp/Desktop/MER_arin/DEAP_data/eda_data/'+str(p)+'_GSR_data_from_DEAP.csv'
            print(file_eda)
            eda_sig = pd.read_csv(file_eda,sep=',', header = None, engine='python')
            return eda_sig

        ##===================================================
        # Resp data read from files
        ##===================================================
        def resp_data(p):
            file_resp = '/home/gp/Desktop/MER_arin/DEAP_data/resp_data/'+str(p)+'_Respiration_data_from_DEAP.csv'
            print(file_resp)
            resp_sig = pd.read_csv(file_resp,sep=',', header = None, engine='python')
            return resp_sig
        
        #------------------------------------
        # Once file fetched data stored here 
        #------------------------------------
        grand_eeg = eeg_data(p)
        grand_eda = eda_data(p)
        grand_resp = resp_data(p)

        
        for jj in range(0,videos):
            v = jj+1 #Video number
            print('=========================================================================')
            p_v = 'Person:'+ ' ' +str(p)+ ' ' +'Video:'+str(v)
            print(p_v)
            
            emotion_label =[]
            
            
            ##===================================================
            # Data read from files
            ##===================================================
            eeg_sig = grand_eeg.loc[grand_eeg.iloc[:,1] == v]
            eda_sig = grand_eda.loc[grand_eda.iloc[:,1] == v]
            resp_sig = grand_resp.loc[grand_resp.iloc[:,1] == v]
            
            #=================================================
            #emotion labels (valence, arousal) mapping 0-1
            #=================================================
            val = eeg_sig.iloc[0,8067]
            aro = eeg_sig.iloc[0,8068]
            
            #valence emotion maping 0-> low valence and 1-> high valence

            if (val >5):
                vl = 1 #high valence
            else:
                vl = 0 #low valence

            #arousal emotion maping 0-> low arousal and 1-> high high arousal
            if (aro >5):
                al = 1 #high arousal
            else:
                al = 0 #low arousal
                
            y_act_val = np.array([vl])
            y_act_aro = np.array([al]) 
            
            #==========================================================
            # Predicted Valence and Arousal labels list initialization
            #==========================================================
            eeg_val_prdt=[]
            eeg_aro_prdt =[]
            
            eda_val_prdt=[]
            eda_aro_prdt =[]
            
            resp_val_prdt=[]
            resp_aro_prdt =[]
            
            
            #=========================================
            # Sliding window starts here 
            #=========================================
            slider_eeg = Slider(bucket_size,overlap_count)
            slider_eda = Slider(bucket_size,overlap_count)
            slider_resp = Slider(bucket_size,overlap_count)
            
            eeg_sig = np.array(eeg_sig.iloc[range(0,32),range(3,8067)]) #keeping only eeg signals
            eda_sig = np.array(eda_sig.iloc[:,range(3,8067)]) #keeping only eda signals
            resp_sig = np.array(resp_sig.iloc[:,range(3,8067)]) #keeping only resp signals
            
            slider_eeg.fit(eeg_sig)
            slider_eda.fit(eda_sig)
            slider_resp.fit(resp_sig)

            while True:
                window_data_eeg = slider_eeg.slide()
                window_data_eda = slider_eda.slide() 
                window_data_resp = slider_resp.slide() 
                
                #=================================================
                # Feature extraction from EEG
                #=================================================
                features_eeg = eeg_features(window_data_eeg)
                eeg = np.array([features_eeg])  #EEG raw feature vector
                x_eeg = preprocessing.normalize(eeg) # EEG normalized features [0,1] 
                
                
                #=================================================
                # Feature extraction from EDA
                #=================================================
                eda_features = extract_eda_features(np.array(window_data_eda))
                eda = np.array([eda_features]) #EDA raw feature vector
                x_eda = preprocessing.normalize(eda) #EDA normalized features
                
                #=================================================
                # Feature extraction from Resp belt
                #=================================================

                resp_features = extract_resp_belt_features(np.array(window_data_resp))
                resp = np.array([resp_features]) #RESP BELT raw feature vector
                x_resp = preprocessing.normalize(resp) #RESP BELT normalized features
            
            
                #===================================================
                # Model initialization
                #===================================================
                if init_m == 0:
                    print('EEG Feature shape{}:'.format(x_eeg.shape))
                    print('EDA Feature shape{}:'.format(x_eda.shape))
                    print('RESP BELT Feature shape{}:'.format(x_resp.shape))

                    #========================
                    # For EEG data model
                    #========================
                    eeg_model_val = create_model(x_eeg)

                    eeg_model_aro = create_model(x_eeg)

                    #========================
                    # For EDA data model
                    #========================
                    eda_model_val = create_model(x_eda)

                    eda_model_aro = create_model(x_eda)

                    #==============================
                    # For Resp Belt data Model
                    #==============================
                    resp_model_val = create_model(x_resp)

                    resp_model_aro = create_model(x_resp)

                    init_m = init_m+1


                #===============================================================
                # Emotion Classification --> Valence and Arousal
                #===============================================================

                if c == 0: #For the first time model will return 0 or None
                    tmp_eeg_val = np.array([0])
                    tmp_eeg_aro = np.array([0])
                    
                    tmp_eda_val = np.array([0])
                    tmp_eda_aro = np.array([0])
                    
                    tmp_resp_val = np.array([0])
                    tmp_resp_aro = np.array([0])
                    
                    
                    y_pred_val_eeg = np.array([2])
                    y_pred_aro_eeg = np.array([2])
                    
                    y_pred_val_eda = np.array([2])
                    y_pred_aro_eda = np.array([2])
                    
                    y_pred_val_resp = np.array([2])
                    y_pred_aro_resp = np.array([2])
                    
                    eeg_model_val.fit(np.array([x_eeg]) , y_act_val, epochs = 1, batch_size = 1, verbose=0)
                    eeg_model_aro.fit(np.array([x_eeg]), y_act_aro, epochs = 1, batch_size = 1, verbose=0)
                    
                    eda_model_val.fit(np.array([x_eda]) , y_act_val, epochs = 1, batch_size = 1, verbose=0)
                    eda_model_aro.fit(np.array([x_eda]), y_act_aro, epochs = 1, batch_size = 1, verbose=0)
                    
                    resp_model_val.fit(np.array([x_resp]) , y_act_val, epochs = 1, batch_size = 1, verbose=0)
                    resp_model_aro.fit(np.array([x_resp]), y_act_aro, epochs = 1, batch_size = 1, verbose=0)
                    
                    c=c+1
                    
                else: 
                    
                    tmp_eeg_val  = eeg_model_val.predict(np.array([x_eeg]))
                    tmp_eeg_aro = eeg_model_aro.predict(np.array([x_eeg]))
                    
                    tmp_eda_val = eda_model_val.predict(np.array([x_eda]))
                    tmp_eda_aro  = eda_model_aro.predict(np.array([x_eda]))
                    
                    tmp_resp_val = resp_model_val.predict(np.array([x_resp]))
                    tmp_resp_aro = resp_model_aro.predict(np.array([x_resp]))
                    
                    #Binary output scaling 1 if x > 0.5 else 0
                    y_pred_val_eeg = [1 if tmp_eeg_val > 0.5 else 0]
                    y_pred_aro_eeg = [1 if tmp_eeg_aro > 0.5 else 0]
                    
                    y_pred_val_eda = [1 if tmp_eda_val > 0.5 else 0]
                    y_pred_aro_eda = [1 if tmp_eda_aro > 0.5 else 0]
                    
                    y_pred_val_resp = [1 if tmp_resp_val > 0.5 else 0]
                    y_pred_aro_resp = [1 if tmp_resp_aro > 0.5 else 0]
                    
                    #Fit the model once
                    eeg_model_val.fit(np.array([x_eeg]), y_act_val, epochs = 1, batch_size = 1, verbose=0)
                    eeg_model_aro.fit(np.array([x_eeg]), y_act_aro, epochs = 1, batch_size = 1, verbose=0)
                    
                    eda_model_val.fit(np.array([x_eda]), y_act_val, epochs = 1, batch_size = 1, verbose=0)
                    eda_model_aro.fit(np.array([x_eda]), y_act_aro, epochs = 1, batch_size = 1, verbose=0)
                    
                    resp_model_val.fit(np.array([x_resp]), y_act_val, epochs = 1, batch_size = 1, verbose=0)
                    resp_model_aro.fit(np.array([x_resp]), y_act_aro, epochs = 1, batch_size = 1, verbose=0)
                    
                    

                #===========================================
                # Performance matric update
                #===========================================
                
                #Valence classification EEG

                eeg_acc_val = eeg_acc_val.update(y_act_val[0], y_pred_val_eeg[0])  # update the accuracy metric

                eeg_f1m_val = eeg_f1m_val.update(y_act_val[0], y_pred_val_eeg[0]) #update f1 measure metric
                
                eeg_roc_val = eeg_roc_val.update(y_act_val[0], y_pred_val_eeg[0])
                
                eeg_mcc_val = eeg_mcc_val.update(y_act_val[0], y_pred_val_eeg[0])
                
                #Arousal classification EEG 

                eeg_acc_aro = eeg_acc_aro.update(y_act_aro[0], y_pred_aro_eeg[0])  # update the accuracy metric

                eeg_f1m_aro = eeg_f1m_aro.update(y_act_aro[0], y_pred_aro_eeg[0]) #update f1 measure metric
                
                eeg_roc_aro = eeg_roc_aro.update(y_act_aro[0], y_pred_aro_eeg[0])
                
                eeg_mcc_aro = eeg_mcc_aro.update(y_act_aro[0], y_pred_aro_eeg[0])


                #Valence classification EDA

                eda_acc_val = eda_acc_val.update(y_act_val[0], y_pred_val_eda[0])  # update the accuracy metric

                eda_f1m_val = eda_f1m_val.update(y_act_val[0], y_pred_val_eda[0]) #update f1 measure metric
                
                eda_roc_val = eda_roc_val.update(y_act_val[0], y_pred_val_eda[0])
                
                eda_mcc_val = eda_mcc_val.update(y_act_val[0], y_pred_val_eda[0])
                

                #Arousal classification EDA
                
                eda_acc_aro = eda_acc_aro.update(y_act_aro[0], y_pred_aro_eda[0])  # update the accuracy metric

                eda_f1m_aro = eda_f1m_aro.update(y_act_aro[0], y_pred_aro_eda[0]) #update f1 measure metric
                
                eda_roc_aro = eda_roc_aro.update(y_act_aro[0], y_pred_aro_eda[0])
                
                eda_mcc_aro = eda_mcc_aro.update(y_act_aro[0], y_pred_aro_eda[0])

                #Valence classification Resp Belt

                resp_acc_val = resp_acc_val.update(y_act_val[0], y_pred_val_resp[0])  # update the accuracy metric

                resp_f1m_val = resp_f1m_val.update(y_act_val[0], y_pred_val_resp[0]) #update f1 measure metric
                
                resp_roc_val = resp_roc_val.update(y_act_val[0], y_pred_val_resp[0])
                
                resp_mcc_val = resp_roc_val.update(y_act_val[0], y_pred_val_resp[0])
                

                #Arousal classification Resp Belt

                resp_acc_aro = resp_acc_aro.update(y_act_aro[0], y_pred_aro_resp[0])  # update the accuracy metric

                resp_f1m_aro = resp_f1m_aro.update(y_act_aro[0], y_pred_aro_resp[0]) #update f1 measure metric
                
                resp_roc_aro = resp_roc_aro.update(y_act_aro[0], y_pred_aro_resp[0])
                
                resp_mcc_aro = resp_mcc_aro.update(y_act_aro[0], y_pred_aro_resp[0])
                

                if slider_eeg.reached_end_of_list(): break
                
            #===================================================
            #   Error storing and comparing Part of each models
            #===================================================
            
            eeg_val_MSE = 0.5 * np.square(np.subtract(y_act_val[0],tmp_eeg_val))[0][0][0]
            eeg_aro_MSE = 0.5 * np.square(np.subtract(y_act_aro[0],tmp_eeg_aro))[0][0][0]
            
            eda_val_MSE = 0.5 * np.square(np.subtract(y_act_val[0],tmp_eda_val))[0][0][0]
            eda_aro_MSE = 0.5 * np.square(np.subtract(y_act_aro[0],tmp_eda_aro))[0][0][0]
            
            resp_val_MSE = 0.5 * np.square(np.subtract(y_act_val[0],tmp_resp_val))[0][0][0]
            resp_aro_MSE = 0.5 * np.square(np.subtract(y_act_aro[0],tmp_resp_aro))[0][0][0]
            
            val_MSE = [eeg_val_MSE, eda_val_MSE, resp_val_MSE] #Valence errors 
            aro_MSE = [eeg_aro_MSE, eda_aro_MSE, resp_aro_MSE] #Arousal errors
            

            
            #========================================================
            # Valence Arousal MSE Compariosn and storing in Cbest and Cworst
            #========================================================
            
            if (jj==0): #First Video
                cBest_val = val_MSE
                cBest_aro = aro_MSE
                
                cWorst_val  = val_MSE
                cWorst_aro = aro_MSE
                
            else:
                #-----------------------------------------
                #Beta for calence and arousal calculation
                #-----------------------------------------              
                beta_val = np.true_divide(list(np.subtract(cBest_val,val_MSE)), (1+np.exp(list(np.subtract(cWorst_val,cBest_val)))))
                beta_aro = np.true_divide(list(np.subtract(cBest_aro,aro_MSE)), (1+np.exp(list(np.subtract(cWorst_aro,cBest_aro)))))
                
                #--------------------------------------------------------------
                # Classifiers best MSE error and worst MSE error value update
                #-------------------------------------------------------------
                cBest_val = np.minimum(cBest_val, val_MSE)
                cBest_aro = np.minimum(cBest_aro, aro_MSE)
                cWorst_val = np.maximum(cWorst_val, val_MSE)
                cWorst_aro = np.maximum(cWorst_aro, aro_MSE)
            ##=============================
            # Confusion Matric Calculation
            ##=============================
            
            eeg_cm_val = eeg_cm_val.update(y_act_val[0], y_pred_val_eeg[0])
            eeg_cm_aro = eeg_cm_aro.update(y_act_aro[0], y_pred_aro_eeg[0])
            
            eda_cm_val = eda_cm_val.update(y_act_val[0], y_pred_val_eda[0])
            eda_cm_aro = eda_cm_aro.update(y_act_aro[0], y_pred_aro_eda[0])
            
            resp_cm_val = resp_cm_val.update(y_act_val[0], y_pred_val_resp[0])
            resp_cm_varo = resp_cm_aro.update(y_act_aro[0], y_pred_aro_resp[0])            
            
            #=============================================================
            # Storing All Results
            #=============================================================
            
            emotion_label.append([y_pred_val_eeg[0],y_pred_aro_eeg[0]]) #appending valence & arousal predicted EEG
            emotion_label.append([y_pred_val_eda[0], y_pred_aro_eda[0]]) #appending valence & arousal predicted EDA
            emotion_label.append([y_pred_val_resp[0], y_pred_aro_resp[0]]) #appending valence & arousal predicted resp

            emotion_label = np.array(emotion_label)
            
                    
            #====================================================================
            # Decision label ensemble --> Reward Penalty Based Weightrd Ensemble
            #====================================================================
            val_label = emotion_label[:,0]
            aro_label = emotion_label[:,1]
            
            #------------------------------------------
            # Valence Class ensemble
            #------------------------------------------
            
            p_val = np.dot([w_val],val_label)
            y_prdt_mer_val = (p_val -0.5)
            if(y_prdt_mer_val > 0):
                mer_val = 1
            else:
                mer_val = 0

            #============================================
            # Weight update for Valence
            #============================================
            w_val = w_val + list(np.multiply(w_val,beta_val))    
            w_val_sum = sum(w_val) #total sum of weights
            w_val = np.array(w_val/w_val_sum) #weight rescaling


            
            #------------------------------------------
            # Arousal Class ensemble
            #------------------------------------------            
            p_val = np.dot([w_aro],aro_label)
            y_prdt_mer_aro = (p_val-0.5)
            if(y_prdt_mer_aro > 0):
                mer_aro = 1
            else:
                mer_aro = 0

            w_aro = w_aro + list(np.multiply(w_aro,beta_aro))
            w_aro_sum = sum(w_aro) #total sum of weights
            w_aro = np.array(w_aro/w_aro_sum) #weight rescaling
            
            #========================================================
            # ReMECS performance metric 
            #========================================================
            mer_acc_val = mer_acc_val.update(y_act_val[0], mer_val)
            mer_f1m_val = mer_f1m_val.update(y_act_val[0], mer_val)
            mer_roc_val = mer_roc_val.update(y_act_val[0], mer_val)
            mer_mcc_val = mer_mcc_val.update(y_act_val[0], mer_val)
            mer_cm_val  = mer_cm_val.update(y_act_val[0], mer_val)
            
            mer_acc_aro = mer_acc_aro.update(y_act_aro[0], mer_aro)
            mer_f1m_aro = mer_f1m_aro.update(y_act_aro[0], mer_aro)
            mer_roc_aro = mer_roc_aro.update(y_act_aro[0], mer_aro)
            mer_mcc_aro = mer_mcc_aro.update(y_act_aro[0], mer_aro)
            mer_cm_aro  = mer_cm_aro.update(y_act_aro[0], mer_aro) 
            
            
            eeg_emotion.append(np.array([p,v,eeg_acc_val.get(), eeg_f1m_val.get(), eeg_roc_val.get(),eeg_mcc_val.get(),
                                         eeg_acc_aro.get(), eeg_f1m_aro.get(),eeg_roc_aro.get(), eeg_mcc_aro.get(), 
                                         y_act_val[0], y_pred_val_eeg[0], y_act_aro[0], y_pred_aro_eeg[0]]))
            
            eda_emotion.append(np.array([p,v,eda_acc_val.get(), eda_f1m_val.get(), eda_roc_val.get(),eda_mcc_val.get(),
                                         eda_acc_aro.get(), eda_f1m_aro.get(), eda_roc_aro.get(), eda_mcc_aro.get(),
                                         y_act_val[0], y_pred_val_eda[0], y_act_aro[0], y_pred_aro_eda[0]]))
            
            resp_emotion.append(np.array([p,v, resp_acc_val.get(), resp_f1m_val.get(), resp_roc_val.get(), resp_mcc_val.get(),
                                          resp_acc_aro.get(),resp_f1m_aro.get(), resp_roc_aro.get(), resp_mcc_aro.get(), 
                                          y_act_val[0], y_pred_val_resp[0], y_act_aro[0], y_pred_aro_resp[0]]))
            
            mer_emotion.append(np.array([p,v,mer_acc_val.get(), mer_f1m_val.get(), mer_roc_val.get(), mer_mcc_val.get(),
                                         mer_acc_aro.get(), mer_f1m_aro.get(), mer_roc_aro.get(), mer_mcc_aro.get(), 
                                         y_act_val[0], mer_val, y_act_aro[0], mer_aro]))
            print('Actual emotion [valence: {}, arousal: {}]'.format(y_act_val, y_act_aro))
            print('Single modals Predicted emotion')
            print('[valence, arousal]:')
            print(emotion_label)
            
            print('MER Predicted emotion [valence: {}, arousal: {}]'.format(mer_val, mer_aro))
            print('Val MSE',val_MSE)
            print('Aro MSE',aro_MSE)
            # print('------------------------------')
            print('Beta_Val:', beta_val)
            print('Beta_Aro:', beta_aro)
            print('------------------------------')
            print('W Val:', w_val)
            print('W Aro:', w_aro)
            # print('------------------------------')
            # print('CBest Val MSE',cBest_val)
            # print('CBest Aro MSE',cBest_aro)
            # print('------------------------------')
            # print('Cworst Val MSE',cWorst_val)
            # print('Cworst Aro MSE',cWorst_aro)
            # print('==============================================')            
            # print('..................................................')
            # print('EEG')

            # print("Valence Accuracy:{}".format(round(eeg_acc_val.get(),4)))
            # print("Valence F1 score:{}".format(round(eeg_f1m_val.get(),4)))
            # print("Valence ROC score:{}".format(round(eeg_roc_val.get(),4)))
            # print("Valence MCC score:{}".format(round(eeg_mcc_val.get(),4)))
            
            # print("Arousal Accuracy:{}".format(round(eeg_acc_aro.get(),4)))
            # print("Arousal F1 score:{}".format(round(eeg_f1m_aro.get(),4)))
            # print("Arousal ROC score:{}".format(round(eeg_roc_aro.get(),4)))
            # print("Arousal MCC score:{}".format(round(eeg_mcc_aro.get(),4)))

            # print('..................................................')
            # print('EDA')

            # print("Valence Accuracy:{}".format(round(eda_acc_val.get(),4)))
            # print("Valence F1 score:{}".format(round(eda_f1m_val.get(),4)))
            # print("Valence ROC score:{}".format(round(eda_roc_val.get(),4)))
            # print("Valence MCC score:{}".format(round(eda_mcc_val.get(),4)))
            
            # print("Arousal Accuracy:{}".format(round(eda_acc_aro.get(),4)))
            # print("Arousal F1 score:{}".format(round(eda_f1m_aro.get(),4)))
            # print("Arousal ROC score:{}".format(round(eda_roc_aro.get(),4)))
            # print("Arousal MCC score:{}".format(round(eda_mcc_aro.get(),4)))


            # print('..................................................')
            # print('RESP BELT')

            # print("Valence Accuracy:{}".format(round(resp_acc_val.get(),4)))
            # print("Valence F1 score:{}".format(round(resp_f1m_val.get(),4)))
            # print("Valence ROC score:{}".format(round(resp_roc_val.get(),4)))
            # print("Valence MCC score:{}".format(round(resp_mcc_val.get(),4)))
            
            # print("Arousal Accuracy:{}".format(round(resp_acc_aro.get(),4)))
            # print("Arousal F1 score:{}".format(round(resp_f1m_aro.get(),4)))
            # print("Arousal ROC score:{}".format(round(resp_roc_aro.get(),4)))
            # print("Arousal MCC score:{}".format(round(resp_mcc_aro.get(),4)))

            # print('..................................................')
            print('MER model')

            print("Valence Accuracy:{}".format(round(mer_acc_val.get(),4)))
            print("Valence F1 score:{}".format(round(mer_f1m_val.get(),4)))
            print("Valence ROC score:{}".format(round(mer_roc_val.get(),4)))
            print("Valence MCC score:{}".format(round(mer_mcc_val.get(),4)))
            
            print("Arousal Accuracy:{}".format(round(mer_acc_aro.get(),4)))
            print("Arousal F1 score:{}".format(round(mer_f1m_aro.get(),4)))
            print("Arousal ROC score:{}".format(round(mer_roc_aro.get(),4)))
            print("Arousal MCC score:{}".format(round(mer_mcc_aro.get(),4)))
            print('===============================xxxx================================')
            



-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
/home/gp/Desktop/MER_arin/DEAP_data/eeg_data/1_data_DEAP.csv
/home/gp/Desktop/MER_arin/DEAP_data/eda_data/1_GSR_data_from_DEAP.csv
/home/gp/Desktop/MER_arin/DEAP_data/resp_data/1_Respiration_data_from_DEAP.csv
Person: 1 Video:1
EEG Feature shape(1, 1152):
EDA Feature shape(1, 48):
RESP BELT Feature shape(1, 48):
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn (SimpleRNN)       (None, 1, 3)              3468      
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 1, 3)              21        
_________________________________________________________________
dense (Dense)                (None, 1, 1)              4         
Total params: 3,493
Trainable params: 3,493
Non-trainable params: 0
________________________________

Actual emotion [valence: [1], arousal: [0]]
Single modals Predicted emotion
[valence, arousal]:
[[1 1]
 [1 1]
 [1 1]]
MER Predicted emotion [valence: 1, arousal: 1]
Val MSE [0.07911209017038345, 0.08482865989208221, 0.04999963194131851]
Aro MSE [0.2958894968032837, 0.19762177765369415, 0.20186518132686615]
Beta_Val: [-0.00499154 -0.00373672 -0.00602768]
Beta_Aro: [-0.14061381 -0.07308978 -0.07524336]
------------------------------
W Val: [0.34048925 0.34194186 0.31756889]
W Aro: [0.31272433 0.34494151 0.34233416]
MER model
Valence Accuracy:0.6
Valence F1 score:0.75
Valence ROC score:0.375
Valence MCC score:-0.25
Arousal Accuracy:0.8
Arousal F1 score:0.8889
Arousal ROC score:0.5
Arousal MCC score:0.0
===============================xxxx================================
Person: 1 Video:6
Actual emotion [valence: [1], arousal: [0]]
Single modals Predicted emotion
[valence, arousal]:
[[1 1]
 [1 1]
 [1 1]]
MER Predicted emotion [valence: 1, arousal: 1]
Val MSE [0.05840136855840683, 0.06579200

Actual emotion [valence: [0], arousal: [1]]
Single modals Predicted emotion
[valence, arousal]:
[[0 0]
 [0 0]
 [0 0]]
MER Predicted emotion [valence: 0, arousal: 0]
Val MSE [0.035913512110710144, 0.045378729701042175, 0.0505603551864624]
Aro MSE [0.24934491515159607, 0.26757046580314636, 0.25068309903144836]
Beta_Val: [-0.00098029 -0.00193164 -0.01137622]
Beta_Aro: [-0.10156171 -0.11243624 -0.10166102]
------------------------------
W Val: [0.35201649 0.35487139 0.29311212]
W Aro: [0.27169934 0.3689625  0.35933816]
MER model
Valence Accuracy:0.6667
Valence F1 score:0.7059
Valence ROC score:0.6786
Valence MCC score:0.378
Arousal Accuracy:0.7333
Arousal F1 score:0.6667
Arousal ROC score:0.75
Arousal MCC score:0.4725
===============================xxxx================================
Person: 1 Video:16
Actual emotion [valence: [1], arousal: [1]]
Single modals Predicted emotion
[valence, arousal]:
[[0 0]
 [0 0]
 [0 0]]
MER Predicted emotion [valence: 0, arousal: 0]
Val MSE [0.2163238823413

Actual emotion [valence: [1], arousal: [1]]
Single modals Predicted emotion
[valence, arousal]:
[[1 1]
 [1 1]
 [1 1]]
MER Predicted emotion [valence: 1, arousal: 1]
Val MSE [0.024614088237285614, 0.032186735421419144, 0.027904873713850975]
Aro MSE [0.06366027146577835, 0.06994656473398209, 0.07584201544523239]
Beta_Val: [ 0.0029897   0.00345136 -0.00132929]
Beta_Aro: [-0.02176377 -0.02148233 -0.02124093]
------------------------------
W Val: [0.35326562 0.36055953 0.28617485]
W Aro: [0.27187948 0.36607418 0.36204635]
MER model
Valence Accuracy:0.68
Valence F1 score:0.75
Valence ROC score:0.6528
Valence MCC score:0.3056
Arousal Accuracy:0.68
Arousal F1 score:0.7143
Arousal ROC score:0.6763
Arousal MCC score:0.3595
===============================xxxx================================
Person: 1 Video:26
Actual emotion [valence: [1], arousal: [1]]
Single modals Predicted emotion
[valence, arousal]:
[[1 1]
 [1 1]
 [1 1]]
MER Predicted emotion [valence: 1, arousal: 1]
Val MSE [0.01977494731545

Actual emotion [valence: [0], arousal: [1]]
Single modals Predicted emotion
[valence, arousal]:
[[0 1]
 [0 1]
 [0 1]]
MER Predicted emotion [valence: 0, arousal: 1]
Val MSE [0.035220783203840256, 0.04560873657464981, 0.04386315122246742]
Aro MSE [0.06504631787538528, 0.06782735139131546, 0.07147350907325745]
Beta_Val: [-0.00830346 -0.01040405 -0.01105004]
Beta_Aro: [-0.02235943 -0.02055271 -0.01929881]
------------------------------
W Val: [0.3550931  0.36084191 0.28406499]
W Aro: [0.26565394 0.36568583 0.36866023]
MER model
Valence Accuracy:0.6857
Valence F1 score:0.7179
Valence ROC score:0.683
Valence MCC score:0.3734
Arousal Accuracy:0.6857
Arousal F1 score:0.7442
Arousal ROC score:0.6711
Arousal MCC score:0.3671
===============================xxxx================================
Person: 1 Video:36
Actual emotion [valence: [0], arousal: [1]]
Single modals Predicted emotion
[valence, arousal]:
[[0 1]
 [0 1]
 [0 1]]
MER Predicted emotion [valence: 0, arousal: 1]
Val MSE [0.02738315612

...

Actual emotion [valence: [0], arousal: [1]]
Single modals Predicted emotion
[valence, arousal]:
[[0 1]
 [0 1]
 [0 1]]
MER Predicted emotion [valence: 0, arousal: 1]
Val MSE [0.025037508457899094, 0.03290951997041702, 0.03296411782503128]
Aro MSE [0.043088026344776154, 0.047986146062612534, 0.05935497209429741]
Beta_Val: [-0.00492852 -0.00738328 -0.00787812]
Beta_Aro: [-0.00963695 -0.01173733 -0.01551516]
------------------------------
W Val: [0.40947563 0.34619978 0.24432459]
W Aro: [0.24846658 0.36529939 0.38623403]
MER model
Valence Accuracy:0.7103
Valence F1 score:0.7491
Valence ROC score:0.7017
Valence MCC score:0.4091
Arousal Accuracy:0.7175
Arousal F1 score:0.7685
Arousal ROC score:0.7003
Arousal MCC score:0.415
===============================xxxx================================
Person: 31 Video:40
Actual emotion [valence: [0], arousal: [1]]
Single modals Predicted emotion
[valence, arousal]:
[[0 1]
 [0 1]
 [0 1]]
MER Predicted emotion [valence: 0, arousal: 1]
Val MSE [0.02058712

Actual emotion [valence: [1], arousal: [1]]
Single modals Predicted emotion
[valence, arousal]:
[[1 1]
 [1 1]
 [1 1]]
MER Predicted emotion [valence: 1, arousal: 1]
Val MSE [0.10785781592130661, 0.11367177218198776, 0.1148720532655716]
Aro MSE [0.08011917024850845, 0.07987558096647263, 0.08284903317689896]
Beta_Val: [-0.03316651 -0.03366585 -0.03415661]
Beta_Aro: [-0.01667093 -0.01520748 -0.01362588]
------------------------------
W Val: [0.41072104 0.34620229 0.24307668]
W Aro: [0.24986554 0.36560198 0.38453248]
MER model
Valence Accuracy:0.7091
Valence F1 score:0.7477
Valence ROC score:0.7008
Valence MCC score:0.4068
Arousal Accuracy:0.7163
Arousal F1 score:0.768
Arousal ROC score:0.6988
Arousal MCC score:0.4125
===============================xxxx================================
Person: 32 Video:9
Actual emotion [valence: [1], arousal: [0]]
Single modals Predicted emotion
[valence, arousal]:
[[1 1]
 [1 1]
 [1 1]]
MER Predicted emotion [valence: 1, arousal: 1]
Val MSE [0.0833891928195

Actual emotion [valence: [1], arousal: [1]]
Single modals Predicted emotion
[valence, arousal]:
[[1 1]
 [1 1]
 [1 1]]
MER Predicted emotion [valence: 1, arousal: 1]
Val MSE [0.03788885101675987, 0.04265337437391281, 0.04187532886862755]
Aro MSE [0.06349805742502213, 0.06535732001066208, 0.06936267018318176]
Beta_Val: [-0.00229614 -0.00194299 -0.00157263]
Beta_Aro: [-0.0091242  -0.00857911 -0.00738964]
------------------------------
W Val: [0.40868978 0.34720193 0.24410829]
W Aro: [0.24632888 0.36334887 0.39032225]
MER model
Valence Accuracy:0.7099
Valence F1 score:0.7495
Valence ROC score:0.7009
Valence MCC score:0.4074
Arousal Accuracy:0.7154
Arousal F1 score:0.7678
Arousal ROC score:0.6974
Arousal MCC score:0.4104
===============================xxxx================================
Person: 32 Video:19
Actual emotion [valence: [1], arousal: [1]]
Single modals Predicted emotion
[valence, arousal]:
[[1 1]
 [1 1]
 [1 1]]
MER Predicted emotion [valence: 1, arousal: 1]
Val MSE [0.0306610614

Actual emotion [valence: [1], arousal: [0]]
Single modals Predicted emotion
[valence, arousal]:
[[1 1]
 [1 1]
 [1 1]]
MER Predicted emotion [valence: 1, arousal: 1]
Val MSE [0.07960280030965805, 0.08092926442623138, 0.08142279833555222]
Aro MSE [0.2203797996044159, 0.221194326877594, 0.21893611550331116]
Beta_Val: [-0.02390758 -0.02278018 -0.02349719]
Beta_Aro: [-0.08649226 -0.08583034 -0.08354591]
------------------------------
W Val: [0.40679136 0.34862365 0.24458499]
W Aro: [0.24334778 0.36161632 0.3950359 ]
MER model
Valence Accuracy:0.7082
Valence F1 score:0.7483
Valence ROC score:0.6991
Valence MCC score:0.4041
Arousal Accuracy:0.7153
Arousal F1 score:0.7684
Arousal ROC score:0.6966
Arousal MCC score:0.4095
===============================xxxx================================
Person: 32 Video:29
Actual emotion [valence: [0], arousal: [1]]
Single modals Predicted emotion
[valence, arousal]:
[[1 1]
 [1 1]
 [1 1]]
MER Predicted emotion [valence: 1, arousal: 1]
Val MSE [0.1497818529605

Actual emotion [valence: [0], arousal: [1]]
Single modals Predicted emotion
[valence, arousal]:
[[0 1]
 [0 1]
 [0 1]]
MER Predicted emotion [valence: 0, arousal: 1]
Val MSE [0.018721505999565125, 0.023440096527338028, 0.024463769048452377]
Aro MSE [0.01717790775001049, 0.01920265704393387, 0.021474674344062805]
Beta_Val: [0.00160129 0.00204466 0.00178926]
Beta_Aro: [0.00154201 0.00152437 0.00152559]
------------------------------
W Val: [0.40974579 0.34734547 0.24290875]
W Aro: [0.24162093 0.36118086 0.39719821]
MER model
Valence Accuracy:0.7097
Valence F1 score:0.7478
Valence ROC score:0.7016
Valence MCC score:0.4088
Arousal Accuracy:0.7167
Arousal F1 score:0.7706
Arousal ROC score:0.6972
Arousal MCC score:0.4111
===============================xxxx================================
Person: 32 Video:39
Actual emotion [valence: [0], arousal: [1]]
Single modals Predicted emotion
[valence, arousal]:
[[0 1]
 [0 1]
 [0 1]]
MER Predicted emotion [valence: 0, arousal: 1]
Val MSE [0.015744594857

In [17]:
print('EEG Val CM:')
print(eeg_cm_val)

print('EEG aro CM:')
print(eeg_cm_aro)

print('EDA Val CM:')
print(eda_cm_val)

print('EDA aro CM:')
print(eda_cm_aro)

print('RESP Val CM:')
print(resp_cm_val)
print('RESP Aro CM:')
print(resp_cm_aro)

print('ReMECS Val CM:')
print(mer_cm_val)

print('ReMECS aro CM:')
print(mer_cm_aro)

EEG Val CM:
             0      1
      0    369    203
      1    152    556
EEG aro CM:
             0      1
      0    319    224
      1    127    610
EDA Val CM:
             0      1
      0    345    227
      1    154    554
EDA aro CM:
             0      1
      0    309    234
      1    128    609
RESP Val CM:
             0      1
      0    358    214
      1    157    551
RESP Aro CM:
             0      1
      0    295    248
      1    138    599
ReMECS Val CM:
             0      1
      0    358    214
      1    158    550
ReMECS aro CM:
             0      1
      0    308    235
      1    128    609


In [18]:
print('[acc_val,f1m_val,roc_val,mcc_val,acc_aro,f1m_aro,roc_aro,mcc_aro]')
#=============================
# EEG
#=============================
t_eeg_emotion =pd.DataFrame(eeg_emotion) 
t_eeg_emotion = t_eeg_emotion.iloc[:,range(2,10)]
t_eeg_emotion_avg = np.array(np.mean(t_eeg_emotion))
t_eeg_emotion_std = np.array(np.std(t_eeg_emotion))

print('EEG result Average:', t_eeg_emotion_avg)
print('EEG result std:', t_eeg_emotion_std)

#=============================
# EDA
#=============================
t_eda_emotion =pd.DataFrame(eda_emotion) 
t_eda_emotion = t_eda_emotion.iloc[:,range(2,10)]
t_eda_emotion_avg = np.array(np.mean(t_eda_emotion))
t_eda_emotion_std = np.array(np.std(t_eda_emotion))

print('EDA result Average:', t_eda_emotion_avg)
print('EDA result std:', t_eda_emotion_std)
#=============================
# RESP BELT
#=============================
t_resp_emotion =pd.DataFrame(resp_emotion) 
t_resp_emotion = t_resp_emotion.iloc[:,range(2,10)]
t_resp_emotion_avg = np.array(np.mean(t_resp_emotion))
t_resp_emotion_std = np.array(np.std(t_resp_emotion))

print('Resp Belt result Average:', t_resp_emotion_avg)
print('Resp Belt result std:', t_resp_emotion_std)
#=============================
# MER
#=============================
t_mer_emotion =pd.DataFrame(mer_emotion) 
t_mer_emotion = t_mer_emotion.iloc[:,range(2,10)]
t_mer_emotion_avg = np.array(np.mean(t_mer_emotion))
t_mer_emotion_std = np.array(np.std(t_mer_emotion))

print('MER result Average:', t_mer_emotion_avg)
print('MER result std:', t_mer_emotion_std)

[acc_val,f1m_val,roc_val,mcc_val,acc_aro,f1m_aro,roc_aro,mcc_aro]
EEG result Average: [0.67091222 0.71056111 0.66176224 0.32877376 0.66370922 0.70108858
 0.65002442 0.31015077]
EEG result std: [0.02246962 0.02612477 0.03794567 0.04777225 0.02449907 0.04783714
 0.04196601 0.04390643]
EDA result Average: [0.65544287 0.69860548 0.64557338 0.29670947 0.66160291 0.69888029
 0.64748048 0.3052404 ]
EDA result std: [0.02364523 0.02741524 0.03777394 0.04919794 0.02081427 0.04761135
 0.04158941 0.043076  ]
Resp Belt result Average: [0.66315885 0.70474773 0.65190653 0.65190653 0.65562255 0.69423204
 0.64056225 0.29216275]
Resp Belt result std: [0.01702567 0.02207152 0.0347185  0.0347185  0.0223061  0.04907147
 0.04116369 0.04138545]
MER result Average: [0.70880889 0.74376999 0.69926085 0.40537485 0.720447   0.75366106
 0.70518275 0.42593304]
MER result std: [0.02583777 0.02758706 0.04332467 0.05930401 0.01998198 0.03421356
 0.04425591 0.04153973]


In [19]:
time_stamp = datetime.datetime.now()
run =1
algo= 'RNN/' 
filename = "/home/gp/Desktop/MER_arin/mer_codes/12DEAP_RPWE/Results_RPWE_DEAP/"+algo
fname_eeg = filename + 'Run_'+str(run)+'_'+str(time_stamp)+'_all_person_EEG'+'_' +classifier+'_results.csv'
fname_eda = filename + 'Run_'+str(run)+'_'+str(time_stamp)+'_all_person_EDA'+'_' +classifier+'_results.csv'
fname_resp = filename + 'Run_'+str(run)+'_'+str(time_stamp)+'_all_person_RESP_BELT'+'_' +classifier+'_results.csv'
fname_mer = filename + 'Run_'+str(run)+'_'+str(time_stamp)+'_all_person_MER'+'_' +classifier+'_results.csv'

column_names = ['Person', 'Video', 'Acc_val', 'F1_val','roc_val', 'mcc_val','Acc_aro',
                'F1_aro', 'roc_aro', 'mcc_aro', 'y_act_val', 'y_pred_val', 'y_act_aro', 'y_pred_aro']

eeg_emotion = pd.DataFrame(eeg_emotion,columns = column_names)
eda_emotion = pd.DataFrame(eda_emotion,columns = column_names)
resp_emotion = pd.DataFrame(resp_emotion,columns = column_names)
mer_emotion = pd.DataFrame(mer_emotion,columns = column_names)

eeg_emotion.to_csv(fname_eeg)
eda_emotion.to_csv(fname_eda)
resp_emotion.to_csv(fname_resp)
mer_emotion.to_csv(fname_mer)
